A Diffusion "Experiment"
==

This notebook is a simulation of
random parcel motion, and how the resulting
fluxes relate to a diffusion coefficient.
![box_setup.png](box_setup.png "Box Setup")

The setup for our experiment is a rectangular box, in which fluid parcels
are randomly distributed. Each parcel has a concentration which it retains
throughout the experiment. The simulation carries out a random walk with
a given length scale and time scale. The parcel concentrations are
initialized to represent a linear gradient $\frac{\partial C}{\partial x}$.
We are interested in the flux of tracer mass through an imaginary surface
in the middle of the box, perpendicular to the $x$ axis.

As the simulation steps forward in time several quantities are recorded:

 * time $t$
 * the gradient $\frac{\partial C}{\partial x}$
 * the mass of tracer on the *left* side of the imaginary surface. The box is closed
   such that any change in tracer mass on the left side is due to a flux through the
   surface in the middle of the box.

After recording this data from the experiment, we then compare (a) the observed
flux, (b) the diffusion coefficient inferred from this flux, and (c) the diffusion
coefficient we would expect from a simple scaling.

This notebook is a mix of text blocks (this is one), code blocks (just below here), and the output from
running the code blocks.

Housekeeping: load libraries for computation and plotting
--

In [ ]:
# Set up the notebook for computation and live plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh import palettes
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, ColumnDataSource
output_notebook()
# Load code to make our simulation interactive in this notebook.
import ipywidgets as widgets
# Our simulation will use functions from numpy
import numpy as np

print("This is what code output looks like")

In [1]:
%%html
<!-- fix icon for rewind button -->
<style> i.fa-stop:before { content: "❚◀"; } </style>

Define the parameters of the simulation
--

In [ ]:
N=4000 # number of parcels
L_m=1.00 # length scale for random steps
dt=1.0 # time scale for random steps
# dimensions of the box the particles move around in.
box=[ [-50,50], # range of x dimension 
      [-20,20], # range of y dimension
      [-1,1]] # range of z dimension
# We will look at the flux through a surface in the middle of
# the box, perpendicular to the x axis. 
box_size=[ high-low for low,high in box]
box_A=box_size[1]*box_size[2] # area of that surface
box_V=box_size[0]*box_size[1]*box_size[2] # volume of the whole box.
center_x=(box[0][0] + box[0][1])/2.0 # measure fluxes through surface x=center_x

Define the simulation
--

Define the initial simulation state (parcel locations and concentrations),
how the state evolves in time, and how data is extracted at each step
for later analysis

In [ ]:
def initialize():
    # These define the instantaneous state of the simulation
    global parcel_x,parcel_C,t
    # These quantities we record as time progresses
    global mass_on_left, times, gradients, C_center

    # Set the initial conditions:
    t=0.0 # simulation time.
    parcel_x=np.zeros( [N,3], np.float64) # [Nparcels,{x,y,z}] ~ meters
    for dim in [0,1,2]:
        # Parcels randomly distributed throughout the domain
        parcel_x[:,dim]=np.random.uniform( box[dim][0], box[dim][1], size=N)

    # parcel concentration is initialize with the x coordinate to create a 
    # smooth gradient in x.
    parcel_C=box_size[0]+parcel_x[:,0] # [Nparcels] ~ g/m3 (concentration)

    # Simulation will update parcel locations, and at each step
    # calculate these summary quantities:
    mass_on_left=[] # mass of tracer on the left side of the box [Ntimesteps]~g
    times=[]        # simulation time associated with each step. [Ntimesteps]~s
    gradients=[]    # concentration gradient [Ntimesteps]~ (g/m3)/m
    C_center=[]     # concentration at the center of the box.

# The code for our simulation has two tasks:
#  1. Update the state (parcel_x,t)
#  2. Record summary information

def step_forward():
    global parcel_x,t

    # For each direction, add a random jump, uniformly
    # random between [-L,+L]
    for dim in [0,1,2]:
        parcel_x[:,dim]+=np.random.uniform(low=-L_m,high=L_m,size=N)
        
        # Parcels that would leave our box get "bounced"
        # back into the box
        outside=parcel_x[:,dim] - parcel_x[:,dim].clip(*box[dim])
        parcel_x[:,dim]-=2*outside
    t+=dt
    
def record():
    times.append(t)
    C_left=np.mean(parcel_C[ parcel_x[:,0]<center_x])
    V_left=(center_x-box[0][0])*box_size[1]*box_size[2]
    mass_on_left.append( C_left*V_left )
    slope,inter =np.polyfit(parcel_x[:,0],parcel_C,1)
    gradients.append(slope)
    # Estimate center concentration from the 20 nearest parcels
    C_center.append( parcel_C[np.argsort(np.abs(parcel_x[:,0]-center_x))[:20]].mean())

initialize() # Create initial state
record() # Save the initial state.

Plot the simulation state
--

Plot parcels and set up the animation.

In [ ]:
parcel_data=ColumnDataSource(dict(x=parcel_x[:,0],
                                  y=parcel_x[:,1],
                                  C=parcel_C))

# Format current time of the simulation
def title_txt(): return f"t={t:.1f}s"

colors=linear_cmap(field_name='C',low=parcel_C.min(),high=parcel_C.max(),
                   palette=palettes.Plasma256)
p = figure(x_range=box[0], y_range=box[1],plot_width=800, plot_height=300,
           toolbar_location=None,title=title_txt())
r = p.circle(x='x',y='y', radius=box_size[0]*0.005, fill_color=colors, fill_alpha=0.6, 
             line_color=None, source=parcel_data)
color_bar = ColorBar(color_mapper=colors['transform'], width=8,  location=(0,0),
                     title="Conc. (g/m3)")
p.line(x=[center_x,center_x],y=box[1],line_color='black',line_dash='dashed',line_width=2.5)
p.add_layout(color_bar, 'right')

# get an explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

def plot_update(step,length):
    # Update the simulation state for the given step number
    # and length scale
    global L_m
    L_m=length
    if (step==0) and (t>0):
        initialize()
        r.data_source.data['C']=parcel_C
    elif step>0:
        step_forward()
    record()

    r.data_source.data['x']=parcel_x[:,0]
    r.data_source.data['y']=parcel_x[:,1]
    p.title.text=title_txt()
    push_notebook(handle=target)
        
widgets.interact(plot_update,
                 step=widgets.Play(value=0,
                                   min=0, max=10000,
                                   step=1, interval=20),
                 length=widgets.FloatSlider(value=L_m,min=0,max=8.0,
                                        step=0.005,
                                        description='L_m:',
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='.3f'))
;

How did the center concentration evolve over time?
--
We expect concentration to follow the diffusion equation:
$$ \frac{\partial C}{\partial t} = D \frac{\partial^2 C}{\partial x^2}$$

In [ ]:
p3=figure(plot_width=800, plot_height=300,
          y_range=[parcel_C.min(),parcel_C.max()])
p3.line(times,C_center)
p3.xaxis.axis_label='Time (s)'
p3.yaxis.axis_label='Conc. at center (g/m^3)'
show(p3)

Mass and Flux
--
Before getting to flux, how did the tracer mass on the left side evolve?  

In [ ]:
p2 = figure(plot_width=800, plot_height=300)
p2.line(times,mass_on_left)
p2.xaxis.axis_label='Time (s)'
p2.yaxis.axis_label='Mass on left side (g)'
show(p2)

Calculate mean flux and mean gradient
--

Define the flux $J_x$ as positive in the $+x$ direction. If we draw out a mass balance for the left side of the box we can see that the change in tracer mass can only come from a flux through the surface in the center of the box.  From there we can relate flux to $d m_L / dt$: 

$$J_x = - \frac{1}{A} \frac{d m_{L}}{dt}$$

Where $m_L$ is the mass on the left side of the domain.

Calculate both the flux $J_x$ and gradient $\partial C/\partial x$
as an average from $t=0$ to $t=t_{last}$


In [ ]:
delta_m=(mass_on_left[-1] - mass_on_left[0])
delta_t=(times[-1] - times[0])
m_dot=delta_m/delta_t # ~ g/s
flux=-m_dot/box_A # ~ g/s/m2
print(f"flux={flux:.3f} g/(m^2 s) averaged from t={times[0]}s to t={times[-1]}s")

In [ ]:
gradient=np.mean(gradients)
print(f"Average gradient so far: {gradient:.3f} (g/m3)/m")

Estimate $D$ from the experiment
--

Based on our definition of Fickian flux:
$$J_x = -D \frac{\partial C}{\partial x}$$

In [ ]:
Dobs=-flux / gradient
print(f"Observed diffusion coefficient: {Dobs:.3f} m^2/s")

Estimate $D$ by Scaling
--

In [ ]:
u_m=L_m/dt
print(f"Velocity scale: {u_m:.3f}m/s")
Dscaled=u_m*L_m
print(f"Estimated diffusion coefficent from scaling: {Dscaled:7.3f} m^2/s")
print(f"             Observed diffusion coefficient: {Dobs:7.3f} m^2/s")

The sign is correct. Order of magnitude is  ... *close*. 

For scaling relationships and the
types of approximations we will encounter, that is often as good as it gets.

But why are we off by nearly an order of magnitude? How can this be improved?




Fokker-Planck Solution for Random Walks
--

The Fokker-Planck equation is a stochastic differential equation
that links our continuous view of diffusion (and advection) to 
the probability distribution of particle movement. 

For this experiment, we can use a simplified solution to F-P from
Visser:

Assuming a uniform $D$, and a uniform random variable $R \in [-1,1]$,
particle steps along a specific axis are given by:

$$ \Delta x = R  \sqrt{2 r^{-1} D \Delta t} $$ 

where $r$ is the standard deviation of $R$, $r=\sqrt{E[R^2]}=1/3$.

In our "experiment" $\Delta x$ is uniform over $[-L,L]$, 
equivalent to assuming $\sqrt{2r^{-1}D\Delta t}=L$.

In that case, the $D$ that is *actually* simulated is:

$$D = \frac{r}{2} \frac{L^2}{\Delta t} = \frac{D_{scaled}}{6}$$

In [ ]:
Dapp=u_m*L_m/6
print(f"Scaled D:       {Dscaled:7.3f} m^2/s")
print(f"Observed D:     {Dobs:7.3f} m^2/s")
print(f"Approximated D: {Dapp:7.3f} m^2/s")

Still not exact?? Chalk it up to unsteadiness and randomness.

Note that the gradient is changing in time, and it can take very large numbers of particles to even out the noise of a random process. As the number of particles is increased and the walls of the box moved away from our area of interest, the observed and expected diffusion coefficients will converge.


In [ ]:
# Style settings
# !jt -t grade3 -nfs 10 -lineh 115 -ofs 14 -tfs 14 -m 0.1,0.1,0.1,0.1 -cellw 90%